In [64]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [66]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
#op_model = "AV_Model"
#op_model = "AV_IN_Model"
#op_model = "PowerLaw"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"
#op_model = "SocialForce"
#op_model = "SocialForce_Elliptical_A"
#op_model = "SocialForce_Elliptical_B"
op_model = "SocialForce_CollisionPrediction"


include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

#strat_model = "Constant"
strat_model = "Towards_Goal"
#strat_model = "Two_Goals"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

The following parameters were given in zanlungo_SocialForceModel_2011:
[2.0, 1.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.66, 1.71, 0.0, 0.0, 0.0, 0.71, 0.0]
v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ


In [79]:
#### parameters of the agents ####
p = [2.0, 1.7, 0.0, 0.0, 0.3, 0.0, 0.0, 0.1, 1.0, 0.0, 0.0, 0.0, 0.71, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
N = 9
system_size = (20.0, 20.0) #important for periodic boundaries

### of the simulation ###

# simulation time-step #
δt = 0.01
sim_p = [0.0, 10.0, 0.05, δt, p[1]*p[2]] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"


### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = create_geometry_closed_room(10.0, 10.0, 0.3)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

R_ = 5.0
p0 = (10.0, 10.0)
Init_Antipodal!(menge, R_, p0)

In [80]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [81]:
using GR
GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(4,16, 4, 16)
        GR.setviewport(0.0, 1, 0.0, 1)       

        plot_agents(positions[j, :], headings[j, :], 2.1, 100, 0.7, 999, 0.1)

        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.7, 0.95, timer)
        GR.text(0.7, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [82]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAGhGttZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAVV2WIhAAX//73x0/Apuj5ZsudRT0hMIrY+kgovVjs0DJ1AAADAAADACEEKABh6Q3X1x844SGWAADYDTOVYPDq4AxVeOlySpC0fXdHCSaMSrGI8B4XBdE1aCOG9KJn0V4T9i+VFmZWNnJbNkM0LIiJg1NPCjojoaxJ2x/WOyL36jlRkITPMjPG2kkoOlMSe4xraD/JJm/aNJDDQc3WsrnshRDEePDLsoPQeScykNPdzOhTyL0NKByiJ/kRldWxURL+4zu7+Uc2neewoHP4yQYG26igB2atGDm54Jm/Mb/rukWOf8VMdpXOIFcDT7Dcj2Abc6I0p66n53mJ1mJ0tpttY2yEQjfjZFxjzMi9Z0xtX/7oWToJUJPpqaUQS9VJMcEm3MrtLSO5kjduVGYe3cBjsQzTozjdxvyWTP0qeSYHMODrNsmXb85rPfX+5fZ5p9ts8csAwz27s6YQR6a1a3/Mv827c/mwA1yB/lkJM42j9hamgdfUc/Y3+xGff84I4XGoMoYzSuJTIeTyiSvxSx4R6uuflK6nANcBLTMFCuLaNttfJPo3SEMSJt1Tx/////sd6vvCFVRgPbJqq1hjWAl7jURz76g58hS2bx26yagSinSdiwDwuGM4B9hoyk+P9Y1HRfOJy85ULpYxwwvn1aRtV4wmlbQ/ut5whADgmLxfW6Dd77K9C08fVxJqy5KaZ3QRvi1M64IdOQcDk2FPG4N53rnI6a2WfU95+rfJ+J5Qcu+RoJ/5G2O4R6OvuMxYhagBtVdHU4KvenRZEh1a+XBC/vXq+J1MgMP3si/3eVefJ9kG58uVwVR1L9JD2JuK8QrB1rl3HP7tCq5MUGeD19PJbn0kvNUj+poZ0gXbKH56Mm4hzpyqSvUA1qkxp1ct7/lBToW0dX5J775ibSH3QUXB7ZHz3mjEvy99nb8um5nOjOSVO+u4AV2s+R/zeo7DNJUAncnLtjbxdZascxWUmwLerzBf/1BpJYBRSUVZPffkeiWFpB1lAb98ssxyfW+eiNelunxSzehhmR+WNWlJSIrr+k7rzPl0+mzaAQ6imuOi5ie2Cs5fp3aKNwgR6DYAN7ZZK7zRlBWL5MfXzVXDYQFvO5iHQXJRt0CmhAfMb++58815qAeASCv4h9k7Fxwimv/+e+cfeiY2EWFWoVqNgNWWPtDEQHnh+Y/+uTTaNf9ri4mN+XmlWFbmyBsZXpeZSM74vYSeArvVcI6LPSeXCTr7tQK8jzeOkuTwXqFtoKhanaVCSaz9oE2pQ3he2jAylE7C9tgaCz437/FGUJx4Us+4hKjVL0arqGoONT7saDO/yQ3JWonHM0LlImsUOGGjtmIn72BFtTf/55qTQFuBgvS3EabYL2Iw/lSHoq4vWNJY1VBgs0YM3WmTVNh21v2S1rINXq0KIEZQtpC2NKvIRIuFUy9xMb+yvmbBKfYwn3wEB6/31EBFX8cvRSZHhHk6g8F6Xr/3hr4gYHfRUe6sC7qqwoS5IBFZxu+TIkihLBRpFYN86i1Wo+n7bDAiFh9GGxXHQnlfrHaqqHSN3FA61fBKGEWNRP5sEQPVa0Dzl4Jv/1obrvHQaaTvXP0jN05rMTmMEbC0YoY/8wzicrzA5pf98tFfozo1P/7imGGaB34sbdCrpL+z5+YdUFrUMjQhAWeWcoEH4dbuOWi3Ay0T3EQs0LPz/+XIWBnkRNbe1iqVXiSnOquA6+G5AWbsmgmcUu/3zv+h3dx6rBx2x//6d6HZL7wUnUJnpeyh8vqQHufyqKxW/ePOBS+9O+4TRFDDwu0wqlvAM3rc0zDX3bXwK9B3Y8LaUV/yJ8+vx79tyEI2LR3U94eUzsMzM+TmGWOL0EwdhJKQ4sRuDd1n0JjEHWcpVPRacvhpCH1Awr2t+zoPfOYZhJzdKOApyo6iZCtYgUA8G4DLU/PMHOliWDAHSm7LkfDoCGWS7BMG7CuV3vZKAPtrdWhoOvPy4SqrqimLM4T9Vul5SH2ymxXzU1jdYV0u1QeYHTqBe/FrSXwhXA2qHlKnjJsCvqxtChnMjO9k+luxrpHGUp2tcaCZnyLVE6sHV9px0bNVjgoREdjLruKYvVCPfljXgSsSx0FART2NSMhkhAOr1+IkjMLHtm7R+AEulf5jP6VYA7Z8llbOuyRcz5NqS/3Ah0PQhDDkjlTNUqzR84JrUC4BSYzwumX0rjZVLPljJY/XdWnq2Y9TPJOJ8chzyTe5xVEHz+/kcH2B7/++CZkFY2QLH8lMGVq3DnVigqhvADmVkzf/4xCieOseJ6bNZzyHZViwweSv+qN7G+IyBA5kBsbYQwJbExaQJ9BN07I7rtEkgJqhR/3zoNzP6xQXshkCLEtHdpayS549AFV2aL1otnbpvYGhMyKy3BthhlH2XAkLp0mleYeEw6haxDBPkiLuvmCrJ29tOxrbxrKQ9z1yhQvejF1FXyd+qXbXOp2srx+9rRnaUSW2n296bs2v/Kd/tNZBICUVkvsny1dy/OC6t1y/IMD/Qe8412yKadxgWwzNTVFvBAzb0gArNbYiOe7we0XEjWWMj4zerVy2kHfI9dm2462Ng/LS8dNgUWKlRwPcxP50jaMs+7INJIkN6GW431y60c/NJGvujno8sdVAG4gKDcUeL3FmhxK7hLxqOKssE4zK9dXk0NpFQFYVkNzfXnp569vsMlV9xHVv0UgOZRQPd0cm3V60b88iWyCa35p2TWyiPhEEamXHw1mND8BKeEygcOneslhgwchaRlwNT8BFezH1iqJ3sLK/AwqpFJbyMEhs5wQj1GrIHDzG4iLkhZGUsQn3X3zZ8/CNPuost/U6CLKBE/IAmw0/OEkqzkPNZltt+mUFerwjTilOn42TFw/DT8hJQlTKEJW0KlTJ9//z4gHP6h8C00Iy4eYE95P7gOxj3czqB0DNcYgEMKg9b6GCx2z

In [84]:
### To save the results in a .txt file in the format ID/Frame/X/Y ###

Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "Circle_8_AV_IN"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
### Perform Multiple Runs and Save the Output ###

samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [ ]:
### Save Initial Conditions for UMANS Software ###

name = "Circle_8"
path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/agents/"


#Set_Goal_from_E_des!(menge, (99.0, 0.0))
#Change_Coordinate_System_Agents!(menge, -1 .*(system_size./2))
Write_Agents_XML(menge, name, path)

In [ ]:
### Save Scenario for UMANS Software ###

path = "/home/jakob/Dokumente/Renne_Software/UMANS_3/examples/"
policy = "SocialForces"
agents = "Circle_8"

name = string(agents, policy)

Write_Scenario_XML(path, name, sim_p, system_size, policy, agents)